# Working with trajectory ensembles

**Welcome**

Welcome to the MD section of the EncoderMap tutorial. All EncoderMap tutorials are provided as jupyter notebooks, that you can run locally, on binderhub, or even on google colab.


Run this notebook on Google Colab:

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AG-Peter/encodermap/blob/main/tutorials/notebooks_MD/01_Working_with_trajectory_ensembles.ipynb)

Find the documentation of EncoderMap:

https://ag-peter.github.io/encodermap

**Goals:**

In this tutorial you will learn:
- [What CVs are.](#primer)
- [How EncoderMaps' new `SingleTraj` class loads MD data.](#singletraj)
- [How a `SingleTraj` can be associated with CVs.](#load_CVs)

### For Google colab only:

If you're on Google colab, please uncomment these lines and install EncoderMap.

In [ ]:
# !wget https://raw.githubusercontent.com/AG-Peter/encodermap/main/tutorials/install_encodermap_google_colab.sh
# !sudo bash install_encodermap_google_colab.sh

<a id='primer'></a>

## Primer

### Collective Variables

Collective Variables (CVs) are often used to simplify and filter the xyz-Data of MD simulations. They are often employed to make sense of a complex protein (or more general molecular) system by breaking it down into just a few well-defined descriptors. They are similar to reaction coordinates, which are 1-dimensional variables along a reaction pathway but can be higher dimensional. When we think about a receptor-ligand system, the distance between the two species is often used as a reaction coordinate. A set of this distance CV and the relative rotation between receptor and ligand can add much more information and help understand the system. It becomes apparent that clever selection of CVs is an important task that many scientists in different fields face day to day.

<!-- <img src="CV_overview.png" width="900" align="center"> -->

With the tools presented in this notebook you will be able to work in a fluent and natural way with MD trajectories and their associated CV data. In the scope of this work we define CVs as:

**a collection of data that is in one of its dimensions aligned with the frames/timesteps of the underlying simulation data.**

### Example CVs

Here's a list of widely used CVs:

- Distances: This category of CVs can contain a 1-diemsnional scalar value describing the distance between two species in a receptor-ligand system. A 1-dimensional CV of the end-to-end distance of a protein can describe the proteins folding state in an approximate and generalizing manner. Distance CVs can also be higher-dimensional. The pairwise distances between $\mathrm{C_\alpha}$ atoms of a protein with $n$ residues can be captured as a $n \times n$ matrix. A so-called hollow matrix, where all diagonal-elements are zero which is also symmetric. Most often a vector of length $\binom{n}{2}$ is used to describe the pairwise distances in a protein. The distance between a protein and a membrane/interface would also fall into this category.
 
- Angles: Angular CVs lie in a periodic space of $(-\pi, \pi]$ or $(0, 2\pi]$, or $(0, 360]$. The well-known Ramachandran plot uses a protein's $\phi$ and $\psi$ angles to define regions of $psi$-$phi$ combinations which correlate to different secondary structure motifs. Besides the backbone angles other angles can become important in understanding a protein's conformations. Lysine is often modified via acetylation, phosphorylation, methylation, ubiquitylation and it's sidechain angles ($\chi_1$ to $\chi_5$) can be important descriptors in such a system. Pseudo-Dihedral angles lie also in this category. 

- Integer/binary values. If a protein has well-defined states (folded and unfolder) a binary value describing these states could also be a useful CV.

- Values from other calculations and hyperparameters. An example for this category could be the temperature at which the simulation was carried out, when simulations were conducted at multiple temperatures. If phase space sub states were obtained by either using Markov-Chain models, or by using some sort of clustering algorithm (GROMOS), the membership to such cluster could also present CVs.

- Positional values: Maybe even the full position of an atom or a group of atoms could be an important descriptor for a system.

- etc.

#### Raw data

We can view the raw xyz-data of trajectories as a high-dimensional array with a shape of (n_frames, n_atoms, 3). The last axis correxponds to the cartesian coordinates. Indexing this axis will give us the *x*, *y*, and *z* coordinates, respectively.

In [ ]:
# todo: delete
from __future__ import annotations
import plotly.graph_objects as go
import numpy as np


import encodermap as em

%load_ext autoreload
%autoreload 2

In [ ]:
traj = em.load_project('linear_dimers', traj=0)

print(f"First frame, first atom, x-coordinate:\n{traj.xyz[0, 0, 0]=}")
print(f"First 2 frames, first atom, y-coordinates:\n{traj.xyz[:2, 0, 1]=}")
print(f"All frames, all atoms. Only z-coordinates:\n{traj.xyz[..., -1]}")

The 3-dimensions of this data can be used for plotting. However, we don't gain much from these plots.

In [ ]:
em.plot.raw_data_plot(
    traj,
    frame_slice=slice(0, 5001, 1000),
    atom_slice=slice(0, 1500, 150),
)

Adding more context to the raw xyz-data (like size, color, bonds) we can better understand the data. However, we can't really understand how the protein moves from one folding state to another. Hit the "Play" button and have a look at the first 200 frames of the simulation. Can you spot the LEU56-SER57-ASP58-TYR59 residues transition from a turn into an $\alpha$-helix?

In [ ]:
em.plot.ball_and_stick_plot(
    traj,
    subsample=slice(0, 200, 20),
    animation=True,
)

#### Collective variables

That's when collective variables can come in handy. They break down and combine different xyz data to help us understand the grand picture of protein folding.

##### Angles

A Ramachandran plot is a very general description of secondary structure motifs. The test protein does indeed contain $alpha$-helices and $beta$-sheets. We can't really understand the conformation of the protein.

In [ ]:
em.plot.ramachandran_plot(traj, subsample=1000)

A DSSP plot doesn't condense the $phi$ and $psi$ angles into a single plot. Here we can assign secondary structure motifs to certain residues at certain times of the simulation.

In [ ]:
em.plot.dssp_plot(
    traj,
    simplified=False,
    subsample=slice(0, 500),
)

##### Distances

Distances can also be used as useful collective variables. The end-to-end distance can give helpful insights into a proteins general conformation (collapsed vs. extended).

In [ ]:
em.plot.end2end_plot(traj)

### Sharing MD data

To play with MD data we must first obtain them. Instead of running long MD simulations you can just download the datasets from the University of Konstanz' data repository KonDATA. EncoderMap has a convenience function `get_from_kondata()`, that allows you to fetch those files:

In [ ]:
import encodermap as em
%load_ext autoreload
%autoreload 2
output_dir = em.get_from_kondata(
    "linear_dimers",
    mk_parentdir=True,
    silence_overwrite_message=True,
)

<a id='singletraj'></a>

## Classes for working with MD data

After we have obtained some files let us work with Encodermap's `SingleTraj` and `TrajEnsemble` classes. We need some more imports to also plot some data and visualize it.

In [ ]:
# Packages for numerical data
import xarray as xr
import numpy as np

# Packages for MD data
import mdtraj as md
import MDAnalysis as mda

# Packages for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt

# Packages for file operations
from glob import glob
from pathlib import Path
import os
import tempfile

# nice printing
from rich.pretty import pprint

# With nglview you can view structural data in the notebook
# If not installed you should check it out.
try:
    import nglview as nv
except ImportError:
    print("Check out nglview: https://github.com/nglviewer/nglview")

### The new `SingleTraj` class

The `SingleTraj` class is meant as a container to hold a trajectory's xyz coordinates, its topology, and its CVs. This class builds the backbone of the `TrajEnsemble` class which will be looked at later.

#### Initialization

In the background of the `SingleTraj` class the MDTraj package (https://github.com/mdtraj/mdtraj) works its magic, however this class offers more benefits:

- The `SingleTraj` class keeps track of CVs while indexing and slicing.
- The trajectories' xyz data is only loaded when it is actually needed. Most manipuilation can thus be done before creating huge objects in memory.

The `SingleTraj` class can be initialized in many ways:

- From a trajectory file (.xtc, .dcd, .lammpstrj) and a topology file (.gro, .pdb).

In [ ]:
output_dir = Path(output_dir)
traj_from_xtc_file = em.SingleTraj(output_dir / "01.xtc", top=output_dir / "01.pdb")
print(traj_from_xtc_file)

- From an url of the pdb database.

In [ ]:
traj_from_url = em.SingleTraj('https://files.rcsb.org/view/1YUF.pdb')
print(traj_from_url)

- Or by just by providing a pdb code to the `from_pdb_id()` constructor of the `SingleTraj` class.

In [ ]:
traj_from_pdb_id = em.SingleTraj.from_pdb_id('1YUG')
print(traj_from_pdb_id)

- You can also directly load the trajectory from an EncoderMap project. In this case, the trajectory already has CVs.

In [ ]:
# with the load_project() method
traj = em.load_project('linear_dimers', traj=0)
print(traj)

**`SingleTraj`s keep track of the files they have been instantiated from.**

In [ ]:
print(
    f"basename  = {traj_from_xtc_file.basename:<70}{traj_from_url.basename:<70}\n"
    f"traj_file = {traj_from_xtc_file.traj_file:<70}{traj_from_url.traj_file:<70}\n"
    f"top_file  = {traj_from_xtc_file.top_file:<70}{traj_from_url.top_file:<70}\n"
    f"extension = {traj_from_xtc_file.extension:<70}{traj_from_url.extension:<70}\n"
    f"n_frames  = {traj_from_xtc_file.n_frames:<70}{traj_from_url.n_frames:<70}\n"
    f"n_atoms   = {traj_from_xtc_file.n_atoms:<70}{traj_from_url.n_atoms:<70}"
)

#### On demand loading

**Difference between `traj`, `trajectory` and `top`, `topology`**

`traj` and `top` always give `mdtraj.Trajectory` and `mdtraj.Topology`, respectively. They are loaded *on demand* and return the corresponding `mdtraj` object..

`trajectory` and `topology` can be `False` and represent the current *backend* of the TrajEnsemble object.

When instantiated, `SingleTraj` do not load the data from disk. Only when requested.

In [ ]:
print(traj_from_xtc_file.topology)
print(traj_from_xtc_file.top)
print(traj_from_xtc_file.topology)

In [ ]:
print(traj_from_xtc_file.trajectory)
print(traj_from_xtc_file.traj)
print(traj_from_xtc_file.trajectory)

**Loading can be forced**

Using the `load()` method, the trajectory will be loaded. From that point forwards, the xyz data is kept in RAM and can be accessed. The `unload()` function does the reverse and frees up the RAM, but the xyz data can be loaded again. If your RAM is large enough you would not need the `unload()` function, but it is there nonetheless.

In [ ]:
traj_from_xtc_file.load_traj()
print(traj_from_xtc_file.topology)
traj_from_xtc_file.unload()
print(traj_from_xtc_file.topology)

Inside a context manager, the `SingleTraj` is always loaded and unloaded afterwards.

In [ ]:
with traj_from_xtc_file as t:
    print(t.topology)
print(traj_from_xtc_file.topology)

#### Look at trajs with nglview

If nglview is set up, you can take a look at the trajectory with this code:

In [ ]:
view = traj_from_xtc_file.show_traj()
view

#### Duplication of mdtraj

Some methods and attributes are duplicated from `mdtraj`. This allows us to call some `mdtraj` functions on the `SingleTraj` object directly.

In [ ]:
selection = traj_from_xtc_file.select('name CA')
print(selection[:5])
dssp = md.compute_dssp(traj_from_xtc_file)
print(dssp[0, :5].tolist())

In [ ]:
md.compute_center_of_mass(traj_from_xtc_file)[0]

#### Indexing

By indexing a `SingleTraj`, you get another instance of `SingleTraj` containing only one frame.

In [ ]:
frame = traj_from_xtc_file[0]
print(frame)

The indexing of `SingleTraj`s is postponed, meaning, that for certain indexing types (integers, sequences of integers, slices without the step argument) the trajectory data is not loaded from disk. Only, when loaded, are the indexes applied one after the other.

This cell is fast. No file operations are carried out here.

In [ ]:
traj_from_xtc_file.unload()
frame = traj_from_xtc_file[:10][[0, 1]]
print(frame)
print(frame.trajectory)
print(frame.n_frames)

This cell is a bit slower. Here, the trajectory is loaded from file.

In [ ]:
frame.load_traj()
print(frame)

If the index is larger than the number of frames, the `SingleTraj` class will throw an exception.

In [ ]:
traj_from_xtc_file.unload()
frame = traj_from_xtc_file[10_000]
print(frame)

#### Indexing with `.fsel` (frame select)

`SingleTraj`s keep the number of their frames organized. If a `SingleTraj` is loaded, the frames can be represented by a list of integers: `[0, 1, 2, ..., n_frames]`. Indexing these frames with a `[::2]` slice will give the frames `[0, 2, 4, ..., n]`. If you then index the `SingleTraj` with `[2]`, you will receive frame 4 from the original trajectory. Indexing frames by their original number is done with the `fsel[]` selector of the `SingleTraj`.

In [ ]:
traj[::2][2].time

In [ ]:
traj[::2].fsel[4].time

The `fsel[]` selector allows for more selections. All based on the original frame number. Think of it like pandas `.iloc` and `.loc` selectors:

In [ ]:
traj.fsel[[4, 8, 12]].time

This means, that some frames are not accessible, when we do some weird slicing:

In [ ]:
traj[10].fsel[1]

#### Advanced slicing

You can also give a numpy array, a list or even a slice into the indexing.

Indexing with without a stop (`[::5]`) will put the trajectory into memory.

In [ ]:
traj_from_xtc_file.unload()
subsample = traj_from_xtc_file[::2]
print(traj_from_xtc_file.n_frames)
print(subsample)
print(subsample.n_frames)

In [ ]:
traj_from_xtc_file.unload()
subsample = traj_from_xtc_file[[0, 1, 5, 6]]
print(subsample)
print(subsample.n_frames)

**Note:**
Andvanced slicing can result in trajectories with 0 frames in them, or possibly reverse the time axis. Use this feature only if you are sure about what you are doing.

In [ ]:
traj_from_xtc_file.unload()
subsample = traj_from_xtc_file[5:46:3]
print(subsample)
print(subsample.n_frames)

#### Advanced slicing with HDF5

The HDF5 file format (ending wiht the .h5 extension) allows us to directly extract frames and accelerate loading.
We can save a .h5 formatted file from an `SingleTraj` class by calling:

In [ ]:
with tempfile.TemporaryDirectory() as d:
    d = Path(d)
    traj_from_xtc_file.save(d / 'my_traj.h5', overwrite=True)

Loading of .h5 files is similar to all files:

In [ ]:
with tempfile.TemporaryDirectory() as d:
    d = Path(d)
    traj_from_xtc_file.save(d / 'my_traj.h5', overwrite=True)
    traj_h5 = em.SingleTraj(d / 'my_traj.h5')

    print(traj_h5[0])

#### Iteration

The `SingleTraj` class can be used as an iterator in two ways:

- Iterate over the frames with `for frame in traj`.
- Iterate over frame_number and frame with `for frame_no, frame in traj.iterframes()`.

In [ ]:
for frame in traj[::2][:5]:
    print(frame.index)

In [ ]:
for frame_num, frame in traj[::2][:5].iterframes():
    print(frame_num)

### The `TrajEnsemble` class contains multiple `SingleTraj`s

> The trajectory ensemble is everything you’ve always wanted, and more.  Really, it is.  Trajectory ensembles unlock fundamental ideas in statistical mechanics, including connections between equilibrium and non-equilibrium phenomena.
- Daniel M. Zuckerman (https://statisticalbiophysicsblog.org/?p=92#more-92)

EncoderMap tries to implement the idea of a trajectory ensemble with the `TrajectoryEnsemble` class. A container for multiple `SingleTraj`s.

#### Initialization

A trajectory ensemble can be created by providing it multiple trajectory files:

In [ ]:
output_dir = Path(em.get_from_kondata("pASP_pGLU", mk_parentdir=True, silence_overwrite_message=True))

trajs_from_files = em.load(
    [
        output_dir / "asp7.xtc",
        output_dir / "glu7.xtc",
    ],
    [
        output_dir / "asp7.pdb",
        output_dir / "glu7.pdb",
    ],
    common_str=["asp7", "glu7"]
)

In [ ]:
trajs_from_files

The `common_str` argument provides a means to group similar trajectories. It searches for matching substrings in the filename of the provided trajectories and topologies. In the `trajs_from_files`, there are only one `TrajEnsemble` with a single trajectory per `common_str`.

In [ ]:
pprint(trajs_from_files.trajs_by_common_str)

#### `TrajEnsemble`s are not limited to a single topology

Something that EncoderMap does different than packages like `MDTraj` or `MDAnalysis` is that trajectories can be grouped together even when their topologies are different.

In [ ]:
pprint(trajs_from_files.trajs_by_top)

The `trajs_from_files` instance has trajectories with 2 different topologies in it. This feature is meant to represent the expression of mutations in biological systems. Older versions

https://evolution.berkeley.edu/dna-and-mutations/types-of-mutations/

#### Indexing `TrajEnsemble`s

Indexing `TrajEnsemble`s can yield different output types. Indexing with a sinlge `int` will yield the corresponding trajectory.

In [ ]:
trajs_from_files[1]

lists, numpy_arrays and fance slices (`[1:10:2]`) can also be used for indexing. Indexing trajectories by their `traj_num` is done with the `tsel[]` selector.

In [ ]:
trajs = trajs_from_files.copy()
trajs[0].traj_num = 10
trajs[1].traj_num = 20
trajs.tsel[20]

In [ ]:
trajs.tsel[0]

#### `TrajEnsemble`s can be created by adding two `SinlgeTraj`s

In [ ]:
trajs[0] + trajs[1]

#### `TrajEnsemble`s can also be loaded from EncoderMap's projects

In [ ]:
trajs = em.load_project("pASP_pGLU")
trajs

<a id='load_CVs'></a>

## Loading CVs

After learning about the basics of the `SingleTraj` and `TrajEnsemble` class we will come back to collective variables. There are many ways of adding CVs to you trajectories. The easiest would be to provide an already existing numpy array. However, you will be asked to also provide the attribute name (`attr_name`) of the array. With this you could load multiple CV datasets, that differ in ther attribute names. Here's an example:

### From numpy

In [ ]:
# rename the traj to make the following code more readable
traj = traj_from_xtc_file

# random phi/psi angles in a [0, 2pi] interval
random_raman_angles = np.random.random((traj.n_frames, 2 * traj.n_residues)) * 2 * np.pi

# define labels:
phi_angles = [f'phi {i}' for i in range(traj.n_residues)]
psi_angles = [f'psi {i}' for i in range(traj.n_residues)]
raman_labels = [None]*(len(phi_angles)+len(psi_angles))
raman_labels[::2] = phi_angles
raman_labels[1::2] = psi_angles

# load the CV
traj.load_CV(random_raman_angles, 'raman', labels=raman_labels)

# define some integer values (can be cluster memberships)
random_integers_per_frame = np.random.randint(0, 3, size=traj.n_frames)
traj.load_CV(random_integers_per_frame, 'cluster_membership')

These values can be accessed via directly calling their attribute names (so make sure to use valid identifiers).

In [ ]:
print(traj.cluster_membership)

In [ ]:
print(traj.raman)

There's also the attribute `CVs` that is a dict of these collective variables.

In [ ]:
traj.CVs

However, this is not the end. CVs in a `SingleTraj` class are stored as `xarray.Dataset`s. The dataset can be accessed via `_CVs`.

In [ ]:
traj._CVs

**Why xarray?**

The underlying `xarray.Dataset` is intended to make sure "everything is correct". Every value can be accessed via an unambigous identifier.

In [ ]:
traj._CVs['raman'].loc[{'frame_no': 20, 'RAMAN': 'psi 50'}].values

### Slicing with CVs.

Slicing keeps your values where they should be.

In [ ]:
index = np.where(np.array(raman_labels) == 'psi 50')
print(traj[20].raman[index])
print(traj[[0, 5, 10, 20]].raman[:,index])

### Loading from files

CVs can be loaded by providing a string to files. First, let us save some files.

In [ ]:
# save numpy
np.save('raman_file.npy', traj.raman)

# save text
np.savetxt('cluster_membership_file.txt', traj.cluster_membership)

# save full CV dataset as NetCDF
traj._CVs.to_netcdf('full_CV_dataset.nc')

If not providing an `attr_name`, while loading files, the filename will be used:

In [ ]:
traj.load_CV('raman_file.npy')
traj.load_CV('cluster_membership_file.txt')

In [ ]:
print(traj.CVs.keys())

Multiple CVs can be reconstructed from xarray NetCDF files (most end with .nc). If there are conflicts the new data from disk will overwrite the old.

In [ ]:
traj = em.SingleTraj('1am7_corrected.xtc', '1am7_protein.pdb')
print(traj.CVs.keys())
traj.load_CV('full_CV_dataset.nc')
print(traj.CVs.keys())

### Loading with PyEMMA featurizer

We will now use PyEMMA's featurization pipeline (http://emma-project.org/latest/) to load CV data into our trajectory. For this encodermap has its own Version of PyEMMA's featurizer accessible with `em.Featurizer` which can simply be provided to the `SingleTraj` class.

In [ ]:
import encodermap as em
%load_ext autoreload
%autoreload 2
traj = em.SingleTraj('1am7_corrected.xtc', '1am7_protein.pdb')

# instantiate featurizer
feat = em.Featurizer(traj)

# add features
feat.add_backbone_torsions()

# load
traj.load_CV(feat, attr_name='backbone_torsion')

Possible `add_*` features can be found via:

In [ ]:
i = 0
for attr in dir(feat):
    if attr.startswith('add_'):
        help(getattr(feat, attr))
        i += 1
    if i == 2:
        break

The advantages of this method are:

- The same can be done with the `TrajEnsemble` class (more on that later), which is also parallelized.
- Most of the features contain comprehensive labels themselves.

The labels can be accessed via the `.coordinates` attribute of the `SingleTraj`'s `xarray.Dataset`. They are similar to the `attr_names` but without underscores and all caps.

In [ ]:
print(traj._CVs.coords)

In [ ]:
print(traj._CVs.coords['BACKBONETORSIONFEATURE'].values[:10])

Here, it can be seen, that there are some errors on PyEMMA's backbone_torsion feature. The sequence of backbone angles is scrambled.

### Loading with Encodermap Features

Encodermap features inherit from pyemma, but they are better formatted, regarding the labels. They can be loaded via `traj.load_CV('all')` to load all, or via a single string of list of these strings:

In [ ]:
from encodermap.misc.misc import FEATURE_NAMES
print(FEATURE_NAMES.values())

In [ ]:
traj = em.SingleTraj('1am7_corrected.xtc', '1am7_protein.pdb')
traj.load_CV(['central_angles', 'central_dihedrals'])

In [ ]:
print(traj._CVs.coords['CENTRAL_DIHEDRALS'].values)

In [ ]:
print(traj._CVs.coords['CENTRAL_ANGLES'].values)

### Wrtiting custom features No 1

Writing your custom features can be done by subclassing pyemma's features. Required methods and attributes to make your feature work are:

- The class-level attributes `__serialize_version` and `__serialize_fields`
- The methods `__init__`, `describe`, and `transform`.
- The instance attribute `dimension`, which defines the shape of the returned array.

If you want to change the name of the feature, as it appears in the `xarray.Dataset` you can set the attribute `name`.

In the next cell we will define a Feature that provides a random integer to an atom, based on its hash.

In [ ]:
import encodermap as em
from encodermap.loading.features import Feature
import copy

class RandomIntForAtomFeature(Feature):
    # class inherits from encodermap CustomFeature
    # set required class-level variables
    __serialize_version = 0
    __serialize_fields = ('indexes', 'selstr', )
    
    # write an __init__
    def __init__(self, top, selstr='all'):
        """Init of RandomIntoForAtomFeature.
        
        Args:
            top (mdtraj.Topology): The topology to select atoms from.
            
        Keyword Args:
            selstr (str, optional): The string to provide to top.select().
            Defaults to 'all'.
        
        """
        # Copy top to save it from hypothetical changes
        self.top = copy.deepcopy(top)
        
        # define indexes (this is one of the serializable fields,
        # which could be used by pyemma to save a feature to disk.)
        self.indexes = top.select(selstr)
        
        # set dimension
        self.dimension = len(self.indexes)
        
        # inherit missing methods from base
        super().__init__()
        
    def describe(self):
        """This method is not allowed to take any arguments.
        
        This method provides labels.
        
        Returns:
            list: A lsit of str, each str describing one feature.
            
        """
        # In this method we will build a list of str
        # Each str should describe one of our features
        # We assign ints to atoms, so the labels should tell something about the atoms
        getlbl = lambda at: f"atom {at.name:>4}:{at.index:5} {at.residue.name}:{at.residue.resSeq:>4}"
        labels = []
        for i in self.indexes:
            i = self.top.atom(i)
            labels.append(f"Random int for {getlbl(i)}")
        return labels
    
    def transform(self, traj):
        """This method provides values.
        
        Args:
            traj (mdtraj.Trajectory): An mdtraj.Trajectory.
            
        Returns:
            np.ndarray: The values of the features defined in describe.
        
        """
        # Make sure that the returned array has correct shape
        # In general it is a good idea, that this array has the same length as
        # the trajectory has frames
        # In general means, like, ..., always
        values = traj.xyz[:,:,0].astype(int)
        for i in self.indexes:
            values[:,i] = int(str(hash(str(self.top.atom(i))))[-5:])
        return values
    
    @property
    def name(self):
        # define the name of the feature to appear in `SingleTraj._CVs`
        return 'MyAwesomeFeature'

In [ ]:
traj = em.SingleTraj('1am7_corrected.xtc', '1am7_protein.pdb')
print(traj)
featurizer = em.Featurizer(traj)
feat = RandomIntForAtomFeature(traj.top)
for i in feat.describe()[:200:25]:
    print(i)

In [ ]:
featurizer.add_custom_feature(feat)

In [ ]:
traj.load_CV(featurizer)

In [ ]:
traj._CVs.coords['MYAWESOMEFEATURE'].values

### Writing custom features No 2

In this example we will implement a method of calculating a nematic order parameter. This example will be quite different (working with coarse-grained carbon-hydrate chains (so-called telechelics), and not with proteins), but we will work our way through. Here are some references you might consider:

```
@article{mukherjee2012derivation,
  title={Derivation of coarse grained models for multiscale simulation of liquid crystalline phase transitions},
  author={Mukherjee, Biswaroop and Delle Site, Luigi and Kremer, Kurt and Peter, Christine},
  journal={The Journal of Physical Chemistry B},
  volume={116},
  number={29},
  pages={8474--8484},
  year={2012},
  publisher={ACS Publications}
}

@article{flachmuller2021coarse,
  title={Coarse grained simulation of the aggregation and structure control of polyethylene nanocrystals},
  author={Flachm{\"u}ller, Alexander and Mecking, Stefan and Peter, Christine},
  journal={Journal of Physics: Condensed Matter},
  volume={33},
  number={26},
  pages={264001},
  year={2021},
  publisher={IOP Publishing}
}
```

## Saving trajectory and CVs into one file

A trajectory can (with its CVs) saved as one comprehensive file with the `save()` method. What's more: Loading such a file again makes it possible to access any frames and their corresponding CVs almost instantaneously.

In [ ]:
traj = em.SingleTraj('1am7_corrected.xtc', '1am7_protein.pdb')
traj.load_CV('all')
traj.save('1am7_all_CVs.h5')

In [ ]:
new_traj = em.SingleTraj('1am7_all_CVs.h5')
frames = new_traj[[0, 5, 20, 35]]
frames.CentralCartesians.shape

In [ ]:
frames = new_traj[::5]
print(frames.CentralBondDistances.shape)
print(frames._CVs.coords['CENTRALBONDDISTANCES'].values)

However, CVs are deleted, when the number of atoms is altered.

In [ ]:
subset = frames.atom_slice(frames.select('name CA'))

In [ ]:
subset.CVs

In [ ]:
index = np.where(np.array(raman_labels) == 'psi 50')
print(traj[20].raman[index])
print(traj[[0, 5, 10, 20]].raman[:,index])

### Loading from files

CVs can be loaded by providing a string to files. First, let us save some files.

In [ ]:
# save numpy
np.save('raman_file.npy', traj.raman)

# save text
np.savetxt('cluster_membership_file.txt', traj.cluster_membership)

# save full CV dataset as NetCDF
traj._CVs.to_netcdf('full_CV_dataset.nc')

If not providing an `attr_name`, while loading files, the filename will be used:

In [ ]:
traj.load_CV('raman_file.npy')
traj.load_CV('cluster_membership_file.txt')

In [ ]:
print(traj.CVs.keys())

Multiple CVs can be reconstructed from xarray NetCDF files (most end with .nc). If there are conflicts the new data from disk will overwrite the old.

In [ ]:
traj = em.SingleTraj('1am7_corrected.xtc', '1am7_protein.pdb')
print(traj.CVs.keys())
traj.load_CV('full_CV_dataset.nc')
print(traj.CVs.keys())

### Loading with PyEMMA featurizer

We will now use PyEMMA's featurization pipeline (http://emma-project.org/latest/) to load CV data into our trajectory. For this encodermap has its own Version of PyEMMA's featurizer accessible with `em.Featurizer` which can simply be provided to the `SingleTraj` class.

In [ ]:
import encodermap as em
%load_ext autoreload
%autoreload 2
traj = em.SingleTraj('1am7_corrected.xtc', '1am7_protein.pdb')

# instantiate featurizer
feat = em.Featurizer(traj)

# add features
feat.add_backbone_torsions()

# load
traj.load_CV(feat, attr_name='backbone_torsion')

Possible `add_*` features can be found via:

In [ ]:
i = 0
for attr in dir(feat):
    if attr.startswith('add_'):
        help(getattr(feat, attr))
        i += 1
    if i == 2:
        break

The advantages of this method are:

- The same can be done with the `TrajEnsemble` class (more on that later), which is also parallelized.
- Most of the features contain comprehensive labels themselves.

The labels can be accessed via the `.coordinates` attribute of the `SingleTraj`'s `xarray.Dataset`. They are similar to the `attr_names` but without underscores and all caps.

In [ ]:
print(traj._CVs.coords)

In [ ]:
print(traj._CVs.coords['BACKBONETORSIONFEATURE'].values[:10])

Here, it can be seen, that there are some errors on PyEMMA's backbone_torsion feature. The sequence of backbone angles is scrambled.

### Loading with Encodermap Features

Encodermap features inherit from pyemma, but they are better formatted, regarding the labels. They can be loaded via `traj.load_CV('all')` to load all, or via a single string of list of these strings:

In [ ]:
from encodermap.misc.misc import FEATURE_NAMES
print(FEATURE_NAMES.values())

In [ ]:
traj = em.SingleTraj('1am7_corrected.xtc', '1am7_protein.pdb')
traj.load_CV(['central_angles', 'central_dihedrals'])

In [ ]:
print(traj._CVs.coords['CENTRAL_DIHEDRALS'].values)

In [ ]:
print(traj._CVs.coords['CENTRAL_ANGLES'].values)

### Wrtiting custom features No 1

Writing your custom features can be done by subclassing pyemma's features. Required methods and attributes to make your feature work are:

- The class-level attributes `__serialize_version` and `__serialize_fields`
- The methods `__init__`, `describe`, and `transform`.
- The instance attribute `dimension`, which defines the shape of the returned array.

If you want to change the name of the feature, as it appears in the `xarray.Dataset` you can set the attribute `name`.

In the next cell we will define a Feature that provides a random integer to an atom, based on its hash.

In [ ]:
import encodermap as em
from encodermap.loading.features import Feature
import copy

class RandomIntForAtomFeature(Feature):
    # class inherits from encodermap CustomFeature
    # set required class-level variables
    __serialize_version = 0
    __serialize_fields = ('indexes', 'selstr', )
    
    # write an __init__
    def __init__(self, top, selstr='all'):
        """Init of RandomIntoForAtomFeature.
        
        Args:
            top (mdtraj.Topology): The topology to select atoms from.
            
        Keyword Args:
            selstr (str, optional): The string to provide to top.select().
            Defaults to 'all'.
        
        """
        # Copy top to save it from hypothetical changes
        self.top = copy.deepcopy(top)
        
        # define indexes (this is one of the serializable fields,
        # which could be used by pyemma to save a feature to disk.)
        self.indexes = top.select(selstr)
        
        # set dimension
        self.dimension = len(self.indexes)
        
        # inherit missing methods from base
        super().__init__()
        
    def describe(self):
        """This method is not allowed to take any arguments.
        
        This method provides labels.
        
        Returns:
            list: A lsit of str, each str describing one feature.
            
        """
        # In this method we will build a list of str
        # Each str should describe one of our features
        # We assign ints to atoms, so the labels should tell something about the atoms
        getlbl = lambda at: f"atom {at.name:>4}:{at.index:5} {at.residue.name}:{at.residue.resSeq:>4}"
        labels = []
        for i in self.indexes:
            i = self.top.atom(i)
            labels.append(f"Random int for {getlbl(i)}")
        return labels
    
    def transform(self, traj):
        """This method provides values.
        
        Args:
            traj (mdtraj.Trajectory): An mdtraj.Trajectory.
            
        Returns:
            np.ndarray: The values of the features defined in describe.
        
        """
        # Make sure that the returned array has correct shape
        # In general it is a good idea, that this array has the same length as
        # the trajectory has frames
        # In general means, like, ..., always
        values = traj.xyz[:,:,0].astype(int)
        for i in self.indexes:
            values[:,i] = int(str(hash(str(self.top.atom(i))))[-5:])
        return values
    
    @property
    def name(self):
        # define the name of the feature to appear in `SingleTraj._CVs`
        return 'MyAwesomeFeature'

In [ ]:
traj = em.SingleTraj('1am7_corrected.xtc', '1am7_protein.pdb')
print(traj)
featurizer = em.Featurizer(traj)
feat = RandomIntForAtomFeature(traj.top)
for i in feat.describe()[:200:25]:
    print(i)

In [ ]:
featurizer.add_custom_feature(feat)

In [ ]:
traj.load_CV(featurizer)

In [ ]:
traj._CVs.coords['MYAWESOMEFEATURE'].values

### Writing custom features No 2

In this example we will implement a method of calculating a nematic order parameter. This example will be quite different (working with coarse-grained carbon-hydrate chains (so-called telechelics), and not with proteins), but we will work our way through. Here are some references you might consider:

```
@article{mukherjee2012derivation,
  title={Derivation of coarse grained models for multiscale simulation of liquid crystalline phase transitions},
  author={Mukherjee, Biswaroop and Delle Site, Luigi and Kremer, Kurt and Peter, Christine},
  journal={The Journal of Physical Chemistry B},
  volume={116},
  number={29},
  pages={8474--8484},
  year={2012},
  publisher={ACS Publications}
}

@article{flachmuller2021coarse,
  title={Coarse grained simulation of the aggregation and structure control of polyethylene nanocrystals},
  author={Flachm{\"u}ller, Alexander and Mecking, Stefan and Peter, Christine},
  journal={Journal of Physics: Condensed Matter},
  volume={33},
  number={26},
  pages={264001},
  year={2021},
  publisher={IOP Publishing}
}
```

## Saving trajectory and CVs into one file

A trajectory can (with its CVs) saved as one comprehensive file with the `save()` method. What's more: Loading such a file again makes it possible to access any frames and their corresponding CVs almost instantaneously.

In [ ]:
traj = em.SingleTraj('1am7_corrected.xtc', '1am7_protein.pdb')
traj.load_CV('all')
traj.save('1am7_all_CVs.h5')

In [ ]:
new_traj = em.SingleTraj('1am7_all_CVs.h5')
frames = new_traj[[0, 5, 20, 35]]
frames.CentralCartesians.shape

In [ ]:
frames = new_traj[::5]
print(frames.CentralBondDistances.shape)
print(frames._CVs.coords['CENTRALBONDDISTANCES'].values)

However, CVs are deleted, when the number of atoms is altered.

In [ ]:
subset = frames.atom_slice(frames.select('name CA'))

In [ ]:
subset.CVs